# A brochure generation software using LLMs


In [1]:
import os #for interacting with the local system 
import requests # for making requests to websites
import json #json capabilities
from typing import List # to label that we expect a variable to be a list
from dotenv import load_dotenv # loading environment variables from the .env untracked hidden folder
from bs4 import BeautifulSoup # for parsing the html webpage
from IPython.display import Markdown, display, update_display # using markdown and adjusting displays
from openai import OpenAI #making API calls from OpenAI

In [2]:
# Initialize and constants
#overwriting the system variables and defining the API key 
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
#This code checks if the api_key has a valid format
if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
#class to work with fetching and formatting of websites
class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
#Defining a system prompt

link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"

link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.u\rl/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
l/goes/here/about"},out page", "url": "https://full.u
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [6]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [7]:
#Here is where the LLM magic occurs
#This function calls an LLM to obtain relevant links from the prompts
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [8]:
get_links("https://wise.com/")

{'links': [{'type': 'about page',
   'url': 'https://wise.com/gb/about/our-story'},
  {'type': 'careers page', 'url': 'https://www.wise.jobs'},
  {'type': 'mission page', 'url': 'https://wise.com/p/our-mission'},
  {'type': 'platform page', 'url': 'https://wise.com/platform/'},
  {'type': 'newsroom', 'url': 'https://newsroom.wise.com/en-UKI/'}]}

In [39]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)  #Get links is doing the call to the API model
    #print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [10]:
print(get_all_details("https://wise.com/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://wise.com/gb/about/our-story'}, {'type': 'company page', 'url': 'https://wise.com/p/our-mission'}, {'type': 'careers page', 'url': 'https://www.wise.jobs'}, {'type': 'newsroom', 'url': 'https://newsroom.wise.com/en-UKI/'}, {'type': 'platform page', 'url': 'https://wise.com/platform/'}]}
Landing page:
Webpage Title:
Wise: The international account | Money without borders | Wise Mexico
Webpage Contents:
Skip to main content
Personal
Platform
Features
Learn how millions of customers move their money globally
Send money
Send large amounts
Pricing
Help
EN
Log in
Register
4.8 ★ on App Store
152K reviews
4.8 ★ on Google Play
1.3M reviews
Send and receive money globally
Make online transfers from Mexico simple and transparent. Move your money from MXN to USD, EUR and more in just a few clicks.
Send money now
Rate guaranteed (24h)
1 USD = 19.2656 MXN
You send exactly
MXN
Sending over 20,000 GBP or equivalent?
We'll discount our fee
R

In [11]:
system_prompt = "You are a charismatic and funny assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [40]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url) #makes an AI call 
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [45]:
def brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))
    return result

In [46]:
brochure("WISE","https://wise.com/")

# Welcome to Wise 🌍💸

## About Us
At Wise (formerly TransferWise), we're on a mission to redefine the way you send, receive, and manage money across borders. With our platform, millions of customers globally move around **$13 billion USD** each month, and we're proud to say we're trusted by many.

We believe in **money without borders**—efficient, transparent, and without hidden fees. Our service allows you to easily convert your money between different currencies, including MXN, USD, and EUR, in just a few clicks.

---

## Features

- **User-Friendly Interface**: Send and receive money in a flash, simplifying online transfers from Mexico.
- **Competitive Pricing**: Enjoy low fees and real exchange rates without the hassle. For large sums over **£20,000 or equivalent**, we discount our fees even further!
- **Guaranteed Rates**: Lock in your rate for 24 hours so you know exactly what you're sending and receiving. 
- **Regulated & Secure**: We're regulated by the Comisión Nacional Bancaria y de Valores in Mexico and advocate for your security with 2-factor authentication and 24/7 monitoring by fraud specialists.

---

## Customer Love ❤️

We’re delighted to have a **4.8 ★** on both the App Store and Google Play from our community of over **1.4 million reviews**! Here’s what our happy customers have to say:

> "Always fast transactions and good fees. An invaluable online bank for those who live outside their own country or are frequent travellers." — **Megan**

> "The best money travel buddy! Wise makes finances easier to deal with instantly." — **Gemma**

We pride ourselves on our stellar customer support—available around the clock via email, phone, and chat. Our team of **thousands of specialists** is always here to help you.

---

## Company Culture 🌟

At Wise, we foster a culture of transparency, innovation, and empathy. We’re a diverse team that believes collaboration and creativity are key drivers of our success. Whether you’re sending money to a friend, paying bills abroad, or managing your finances from another country, we aim to make every transaction a breeze!

- **Diversity**: Our team comprises individuals from various backgrounds, cultures, and skill sets.
- **Innovation**: We thrive on creativity and welcome new ideas that push the boundaries of what's possible in the financial world.
- **Collaboration**: Together, we work towards a common goal—building the best financial platform for global citizens.

---

## Careers at Wise

Join our mission of enabling **money without borders**! At Wise, we offer a wide range of opportunities for talented and passionate individuals looking to make an impact. 

- 🌟 **Flexible Work Environment**: We value work-life balance and offer flexible working arrangements.
- 🚀 **Growth Opportunities**: We believe in your potential and provide avenues for professional growth and skill development.
- 🤝 **Diverse Teams**: Become a part of our global team where you can work alongside talented individuals from all corners of the world.

Looking for a new challenge? **[Check out our careers page!](#)**

---

## Get In Touch

Ready to move your money effortlessly? Download the Wise app today and experience financial freedom at your fingertips!

🔗 [Join Us & Start Transferring Today](#)

---

### Wise: Empowering your financial journey globally.
**Remember: Where there's a will, there's a Wise! 😉**

'# Welcome to Wise 🌍💸\n\n## About Us\nAt Wise (formerly TransferWise), we\'re on a mission to redefine the way you send, receive, and manage money across borders. With our platform, millions of customers globally move around **$13 billion USD** each month, and we\'re proud to say we\'re trusted by many.\n\nWe believe in **money without borders**—efficient, transparent, and without hidden fees. Our service allows you to easily convert your money between different currencies, including MXN, USD, and EUR, in just a few clicks.\n\n---\n\n## Features\n\n- **User-Friendly Interface**: Send and receive money in a flash, simplifying online transfers from Mexico.\n- **Competitive Pricing**: Enjoy low fees and real exchange rates without the hassle. For large sums over **£20,000 or equivalent**, we discount our fees even further!\n- **Guaranteed Rates**: Lock in your rate for 24 hours so you know exactly what you\'re sending and receiving. \n- **Regulated & Secure**: We\'re regulated by the Comi

In [16]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [17]:
stream_brochure("Wise", "https://wise.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://wise.com/gb/about/our-story'}, {'type': 'careers page', 'url': 'https://www.wise.jobs'}, {'type': 'mission page', 'url': 'https://wise.com/p/our-mission'}, {'type': 'platform page', 'url': 'https://wise.com/platform/'}, {'type': 'newsroom', 'url': 'https://newsroom.wise.com/en-UKI/'}]}



# Welcome to Wise: Your Global Money Movement Partner! 🌍💸

### Who We Are
Wise is revolutionizing how you move money across borders. Whether you're a student in a foreign country, a digital nomad, or just trying to send a little cash back home, we’re your go-to solution! With a fantastic 4.8 ★ rating on both the App Store and Google Play, our users love us—and you will too!

### What We Do
- **Send Money**: Transfer easily from Mexico to over 70 countries, converting MXN to USD, EUR, and more in just a few clicks.
- **Transparent Pricing**: Say goodbye to hidden fees! With low rates and guaranteed conversions, what you see is what you get.
- **Fast Transfers**:  Millions trust us to move around $13 billion globally each month. Your money arrives at the speed of light (or at least a decent WiFi connection).

### Customer Testimonials 💬
- “Wise has made life across continents possible. Transfers are simple and fast.” - **Stuart**
- “Invaluable for those who live outside their country or travel frequently.” - **Megan**
- “The best money travel buddy!” - **Gemma**
- “A lifesaver for me as a student abroad.” - **Stefani**

### Company Culture
At Wise, we believe in creating **money without borders**. Our workforce comprises 1,000s of passionate specialists, dedicated to ensuring your transactions are secure and seamless 24/7. We foster a collaborative environment where every team member plays a critical role in making international money transfers a breeze!

### Join Us!
Interested in a career where your contributions matter? We're always on the lookout for talented and motivated individuals who want to help us enhance global finance. Our culture champions flexibility, diversity, and a focus on growth—both personally and professionally.

### Why Choose Wise?
- **Safety First**: Your account is protected by advanced security features like biometric authentication and 24/7 fraud monitoring.
- **Unmatched Support**: Our customer service team is here to help, whether it’s day or night, via email, phone, or chat.
- **Regulated**: Wise is fully regulated by the Comisión Nacional Bancaria y de Valores in Mexico, ensuring your finances are in safe hands.

---

Bring your money into the future with Wise—where moving money across the world is as easy as sending a text! 

Visit our website to get started or download the app and join millions of savvy users who trust Wise to move their money **safely and simply**!

### Contact Us
**Website**: [Wise.com](https://wise.com)  
**Follow Us on Socials**: @Wise  
**Support**: help@wise.com

Let's make your money work for you, no matter where you are in the world! 🌏💳


In [18]:
#Now changing the system prompt to change the "color" of the answer. The LLM will be test to produce the brochure in old (Shakespearan) English. 

In [51]:
system_prompt = "You are an eloquent and funny assistant, you write old english only. You are tasked with analyzing the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits.Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [52]:
brochure("WISE", "https://wise.com/")

```json
{
    "links": [
        {"type": "about page", "url": "https://www.wise.com/about"},
        {"type": "careers page", "url": "https://www.wise.com/careers"}
    ]
}
```
  
# Wise Brochure

## Welcome to Wise

Wise is redefining how people and businesses move their money across borders. With our international account, you can send and receive money globally, make online transfers from Mexico, and exchange currencies effortlessly.

### Why Choose Wise?

- **Simple and Transparent Transactions**: Move your money from MXN to USD, EUR, and more in just a few clicks.
- **Guaranteed Exchange Rates**: Know exactly how much your recipient will get with our guaranteed rates for 24 hours.
- **Low Fees**: Our fees are simple and low compared to traditional banks.

### Trustworthy & Reliable

- **Trusted by Millions**: Join millions of customers globally who move around $13 billion USD each month.
- **Regulated in Mexico**: Wise is regulated by the Comisión Nacional Bancaria y de Valores.
- **24/7 Customer Support**: Our specialists are available anytime via email, phone, and chat.

### Secure Transfers

With thousands of fraud specialists dedicated to keeping your money safe and utilizing security measures like biometrics and 2-factor authentication, you can transfer money worldwide securely.

### Customer Testimonials

- "They make our life split between two continents possible." - Stuart
- "Always fast transactions and good fees." - Megan
- "Wise has been a lifesaver for me as a student in a foreign country." - Stefani

### Get Started with Wise

Join us in building a better way to manage money with low fees and maximum ease. 

[Learn about our mission](https://www.wise.com)

### Download Our App

Get the Wise app to manage your finances easily, anytime, and anywhere.

[Scan to get the Wise app](https://www.wise.com/app)

### Join the Wise Team

Interested in working with us? Explore our available positions on our careers page.

[Visit our careers page](https://www.wise.com/careers)

---

For more information, visit our official website at [Wise](https://www.wise.com).

'```json\n{\n    "links": [\n        {"type": "about page", "url": "https://www.wise.com/about"},\n        {"type": "careers page", "url": "https://www.wise.com/careers"}\n    ]\n}\n```\n  \n# Wise Brochure\n\n## Welcome to Wise\n\nWise is redefining how people and businesses move their money across borders. With our international account, you can send and receive money globally, make online transfers from Mexico, and exchange currencies effortlessly.\n\n### Why Choose Wise?\n\n- **Simple and Transparent Transactions**: Move your money from MXN to USD, EUR, and more in just a few clicks.\n- **Guaranteed Exchange Rates**: Know exactly how much your recipient will get with our guaranteed rates for 24 hours.\n- **Low Fees**: Our fees are simple and low compared to traditional banks.\n\n### Trustworthy & Reliable\n\n- **Trusted by Millions**: Join millions of customers globally who move around $13 billion USD each month.\n- **Regulated in Mexico**: Wise is regulated by the Comisión Naciona

In [ ]:
#Switching to a more regular system prompt

In [23]:
system_prompt = "You are a charismatic and funny assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


# Using a third call to the LLM to translate the previous generated brochue

In [53]:
system_prompt_translation = "You are a translation assistant, you know everylanguage in the world. You are tasked with translating text to a specified language"

In [54]:
def translate_prompt_user(brochure, language):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt_translation},
            {"role": "user", "content": f"Translate the following document into {language}: {brochure}"}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))
    
    
    

In [55]:
wise_brochure = brochure("WISE", "https://wise.com/")

```json
{
    "links": [
        {"type": "about page", "url": "https://www.wise.com/about"},
        {"type": "careers page", "url": "https://www.wise.com/careers"}
    ]
}
```

### Brochure for Wise

#### Welcome to Wise
Wise is the leading platform for easy, fast, and secure international money transfers. With millions of customers globally moving around $13 billion every month, we offer a simple solution to send and receive money without borders.

#### Our Mission
At Wise, we are dedicated to building the best way to move and manage the world's money. Our goal is to minimize fees while maximizing ease and speed for our users in Mexico and around the world.

#### Why Choose Wise?
- **Transparent Pricing:** Send money globally with clear and competitive pricing.
- **User-Friendly App:** Manage your money conveniently with our mobile app available on both App Store and Google Play, rated 4.8 stars.
- **Security Guaranteed:** Our platform is secure with 2-factor authentication, encryption, and the support of thousands of fraud specialists.

#### Contact Us
For any inquiries or support, reach out to our dedicated customer service team available 24/7 via email, phone, and chat.

#### Join Our Team
Are you passionate about helping people manage their money globally? Explore exciting career opportunities at Wise. 

---
For more information, visit our official website: [Wise](https://www.wise.com)

# Svenska (Swedish brochure)

In [56]:
translate_prompt_user(wise_brochure, "Svenska")

```json
{
    "links": [
        {"type": "om sidan", "url": "https://www.wise.com/about"},
        {"type": "karriärsidan", "url": "https://www.wise.com/careers"}
    ]
}
```

### Broschyr för Wise

#### Välkommen till Wise
Wise är den ledande plattformen för enkla, snabba och säkra internationella penningöverföringar. Med miljontals kunder globalt som flyttar omkring 13 miljarder dollar varje månad erbjuder vi en enkel lösning för att skicka och ta emot pengar utan gränser.

#### Vår mission
På Wise är vi dedikerade till att bygga det bästa sättet att flytta och hantera världens pengar. Vårt mål är att minimera avgifter samtidigt som vi maximerar enkelhet och hastighet för våra användare i Mexiko och runt om i världen.

#### Varför välja Wise?
- **Transparent prissättning:** Skicka pengar globalt med tydlig och konkurrenskraftig prissättning.
- **Användarvänlig app:** Hantera dina pengar bekvämt med vår mobilapp som finns både på App Store och Google Play, betygsatt 4,8 stjärnor.
- **Säkerhet garanterad:** Vår plattform är säker med 2-faktors autentisering, kryptering och stöd av tusentals bedrägerispecialister.

#### Kontakta oss
För frågor eller support, kontakta vårt dedikerade kundserviceteam som är tillgängligt dygnet runt via e-post, telefon och chatt.

#### Gå med i vårt team
Är du passionerad om att hjälpa människor hantera sina pengar globalt? Upptäck spännande karriärmöjligheter på Wise.

---
För mer information, besök vår officiella webbplats: [Wise](https://www.wise.com)

# Extending to multi-shot prompting in the First LLM call

In [47]:
## Defining a system prompt

link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"

link_system_prompt += "You should respond in JSON as in this example:\n"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

# Multi-shot examples

link_system_prompt += "\n\nHere are some examples:\n\n"

# Example 1
link_system_prompt += "Links: ['https://www.example.com/products', 'https://www.example.com/company/about-us', 'https://www.example.com/services', 'https://www.example.com/careers/']\n"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://www.example.com/company/about-us"},
        {"type": "careers page", "url": "https://www.example.com/careers/"}
    ]
}
"""

# Example 2
link_system_prompt += "\nLinks: ['https://www.acmecorp.net/investors', 'https://www.acmecorp.net/our-story', 'https://www.acmecorp.net/contact', 'https://www.acmecorp.net/jobs']\n"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://www.acmecorp.net/our-story"},
        {"type": "careers page", "url": "https://www.acmecorp.net/jobs"}
    ]
}
"""

# Example 3
link_system_prompt += "\nLinks: ['https://www.globex.org/en/index.html', 'https://www.globex.org/en/company-profile', 'https://www.globex.org/en/news', 'https://www.globex.org/en/join-us']\n"
link_system_prompt += """
{
    "links": [
        {"type": "company page", "url": "https://www.globex.org/en/company-profile"},
        {"type": "careers page", "url": "https://www.globex.org/en/join-us"}
    ]++++++++
    - +
}
"""

# Now the prompt is ready to be used with a new set of links.
# For example, you would add:
# link_system_prompt += "\nLinks: ['https://www.newcompany.io/blog', 'https://www.newcompany.io/who-we-are', 'https://www.newcompany.io/pricing', 'https://www.newcompany.io/open-positions']\n"

In [48]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}


Here are some examples:

Links: ['https://www.example.com/products', 'https://www.example.com/company/about-us', 'https://www.example.com/services', 'https://www.example.com/careers/']

{
    "links": [
        {"type": "about page", "url": "https://www.example.com/company/about-us"},
        {"type": "careers page", "url": "https://www.example.com/careers/"}
    ]
}

Links: ['https://www.acmecorp.net/investors', 'https://www.acmecorp.net/our-story', 'https://www.acmecorp.net/contact', 'https://www.acmecorp.net/jobs']

{
    "l

In [49]:
def brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))
    return result

In [50]:
brochure('Wise', "https://wise.com/")

```json
{
    "links": [
        {"type": "about page", "url": "https://www.wise.com/about"},
        {"type": "careers page", "url": "https://www.wise.com/careers"}
    ]
}
```

# Wise: The International Account

Welcome to Wise, a revolutionary service designed to make international money transfers easy, fast, and secure. With millions of customers across the globe, we provide a streamlined way to send and receive money without borders.

## Why Choose Wise?

- **Transparency**: Our fees are clearly outlined, ensuring there are no surprises when you send money.
- **Global Reach**: Transfer funds to numerous countries including the USA, Canada, Australia, and many more.
- **Security**: Our platform employs advanced security measures, including biometrics and two-factor authentication, to keep your funds safe.

### Trusted by Millions

Each month, customers move approximately **13 billion USD** around the world with Wise. We are regulated by the **Comisión Nacional Bancaria y de Valores** in Mexico and provide 24/7 customer support via email, phone, and chat.

### Customer Testimonials

- "Always fast transactions and good fees, an invaluable service!" - Megan
- "The best money travel buddy! Wise makes finances easier." - Gemma
- "Wise has been a lifesaver for me as a student in a foreign country." - Stefani

### Get Started!

Experience money without borders. Download the Wise app or visit our website to learn more about our mission and start sending money today!

**Web Ratings:**
- **4.8 ★ on App Store** (152K reviews)
- **4.8 ★ on Google Play** (1.3M reviews)
- **4.3 ★ on Trustpilot** (258,709 reviews)

[Learn more about our services](https://www.wise.com).

---

For more information regarding our company, please refer to our **[About page](https://www.wise.com/about)** and our **[Careers page](https://www.wise.com/careers)**.

'```json\n{\n    "links": [\n        {"type": "about page", "url": "https://www.wise.com/about"},\n        {"type": "careers page", "url": "https://www.wise.com/careers"}\n    ]\n}\n```\n\n# Wise: The International Account\n\nWelcome to Wise, a revolutionary service designed to make international money transfers easy, fast, and secure. With millions of customers across the globe, we provide a streamlined way to send and receive money without borders.\n\n## Why Choose Wise?\n\n- **Transparency**: Our fees are clearly outlined, ensuring there are no surprises when you send money.\n- **Global Reach**: Transfer funds to numerous countries including the USA, Canada, Australia, and many more.\n- **Security**: Our platform employs advanced security measures, including biometrics and two-factor authentication, to keep your funds safe.\n\n### Trusted by Millions\n\nEach month, customers move approximately **13 billion USD** around the world with Wise. We are regulated by the **Comisión Nacional